## Interactively browse STAC footprints

STAC items contain information about the spatio-temporal footprint of the data they catalog. Using [Jupyter Widgets](https://ipywidgets.readthedocs.io/en/latest/#), in particular [`ipywidgets.interact`](https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html) and [`ipyleaflet`](https://ipyleaflet.readthedocs.io/en/latest/index.html), we can quickly build an interactive tool for visualizing item footprints. This can be helpful for understanding the path a satellite takes over earth or debugging strange looking item footprints.

We'll start by grabbing some Sentinel-2 L2A items from the Planetary Computer's STAC API.

In [1]:
import pystac_client

catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1"
)

search = catalog.search(
    collections=["sentinel-2-l2a"],
    datetime="2023-04-01",
    query={"platform": {"eq": "Sentinel-2B"}},
    sortby="datetime",
    max_items=200,
)

items = search.item_collection()

Next, we'll build up our tool. We start with our placeholders:

1. An `ipyleaflet.Map` object
2. An empty `ipyleaflet.GeoJSON` layer, which we add to the map

Then we use `ipywidgets.interact` to browse the list of items. This gives us a dropdown with the list of items, which we can scroll through to select a single item for display.
The body of the function updates the state of our map to set

1. The center of the map to the center of the selected item
2. The `data` for the GeoJSON layer, to display the selected item's footprint


In [2]:
import ipywidgets
import ipyleaflet
import shapely.geometry

m = ipyleaflet.Map(zoom=3)
m.layout.width = "600px"
layer = ipyleaflet.GeoJSON()
m.add(layer)


@ipywidgets.interact(item=items)
def browse(item):
    shape = shapely.geometry.shape(item)
    m.center = tuple(shape.centroid.coords[0])[::-1]

    layer.data = item.geometry
    print(item.id, item.datetime.isoformat())


m

interactive(children=(Dropdown(description='item', options=(<Item id=S2B_MSIL2A_20230401T001159_R059_T11XML_20…

Map(center=[81.43518005032495, -121.3895405831003], controls=(ZoomControl(options=['position', 'zoom_in_text',…

Here's an example of using the browser:

<video controls width="600" src="https://ai4edatasetspublicassets.blob.core.windows.net/assets/pc_video/interact-stac-browser-web.mp4">
</video>